## Import Libraries

In [1]:
# Library for file persistence
import os
import re

# Selenium libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Use chrome driver manager
from webdriver_manager.chrome import ChromeDriverManager as DriverManager

# Custom paginate library
from centrisca import Paginate

from operations import Operation

## Parameter Selection

Now you need to select the parameter that you want to apply to the crowler. These parameters will define where the regions that te system use to obtains the data, and the output directory used to the store the extraction result.  

In [2]:
# The search Region
search_region = 'montreal-lasalle'

# The output directory
output_dir = 'data/html'

There are also the fixed parameters that'll be used to generate the url used to perform the crowler.

In [3]:
paginator = Paginate(operation=Operation.FOR_SALE, search_region=search_region, handle_popup=False)

## Persistence definition

This function provides a way to persist the data in filesystem. To change it just reimplement it and change the destination.

In [4]:
def persistence(page: str, page_id: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    file_name = f"centris_{page_id}.html"
    file_path = os.path.join(output_dir, file_name)

    # Write the HTML content to the file
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(page)

## Perform extraction

In [5]:
# Iterate through the pages
for page_number, page in enumerate(paginator, start=1):
    # content = driver.page_source

    page_id = str(page_number).zfill(7)
    persistence(page, page_id, output_dir)
    
paginator.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".next"}
  (Session info: chrome=132.0.6834.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001013bf0d4 cxxbridge1$str$ptr + 2600792
1   chromedriver                        0x00000001013b79f0 cxxbridge1$str$ptr + 2570356
2   chromedriver                        0x0000000100f583d8 cxxbridge1$string$len + 89376
3   chromedriver                        0x0000000100f9d414 cxxbridge1$string$len + 372060
4   chromedriver                        0x0000000100fd7638 cxxbridge1$string$len + 610176
5   chromedriver                        0x0000000100f9134c cxxbridge1$string$len + 322708
6   chromedriver                        0x0000000100f91f94 cxxbridge1$string$len + 325852
7   chromedriver                        0x000000010138a2